In [ ]:
import mediapipe as mp
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Google Collab

from google.colab import drive
drive.mount('/content/drive')

#### Clear temporary files which we will furthur use

In [ ]:
def clearMesh():
        folder_path = "mesh_frames"

        files = glob.glob(os.path.join(folder_path, '*'))

        for file in files:
                os.remove(file)

#### Image merger

In [ ]:
#function to merge two frames side by side(i.e. input and output frames )

def addFrame(img1,img2):
    r1,c1 = img1.shape[0:2]
    r2,c2 = img2.shape[0:2]

    result = np.zeros((r2,c1+c2,3),dtype=np.uint8)

    # print(result.shape)

    result[:,:c1,:] = img1
    result[:,c1:c1+c2,:] = img2

    return result

In [ ]:

#video file
video_path= #enter the path of your video here
video = cv2.VideoCapture(video_path)

clearMesh()

#mediapipe objects
facemesh = mp.solutions.face_mesh
draw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

landmark1 = []
landmarks = []
face = facemesh.FaceMesh(
    static_image_mode=True,
    min_tracking_confidence=0.6,
    min_detection_confidence=0.6,
    refine_landmarks=True
)

#temporary folder
output_folder = 'mesh_frames'
os.makedirs(output_folder, exist_ok=True)

#process each frame
while True:
    ret, frame = video.read()

    if not ret:
        print('Video processing complete.')
        break  

    height, width, channels = frame.shape

    #mediapipe requires RGB format
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


    #landmarks acquired in op
    op = face.process(rgb)
    
    #if no face detected
    if not op.multi_face_landmarks:
        continue

    #extracting face landmarks
    if op.multi_face_landmarks:
        for i in op.multi_face_landmarks:
            if(i.landmark[0] is not None and i.landmark[1] is not None):

                landmarks = []
                landmarks.append(i.landmark[0].y * 480)
                landmarks.append(i.landmark[1].x * 640)
                landmarks.append(i.landmark[2].z)
                landmark1.append(landmarks)

                #Landmarks in the original frame
                # draw.draw_landmarks(
                #     image=frame,
                #     landmark_list=i,
                #     connections=facemesh.FACEMESH_TESSELATION,
                #     landmark_drawing_spec=None,
                #     connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                # )
                # draw.draw_landmarks(
                #     image=frame,
                #     landmark_list=i,
                #     connections=facemesh.FACEMESH_CONTOURS,
                #     landmark_drawing_spec=None,
                #     connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                # )
                # draw.draw_landmarks(
                #     image=frame,
                #     landmark_list=i,
                #     connections=facemesh.FACEMESH_IRISES,
                #     landmark_drawing_spec=None,
                #     connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style()
                # )

                #cv2.imshow("Original Video", frame)

                frame_filename = f"{output_folder}/frame_{int(video.get(cv2.CAP_PROP_POS_FRAMES))}.png"


                #separate window for the face mesh
                mesh_window = np.zeros_like(frame)
                draw.draw_landmarks(
                    image=mesh_window,
                    landmark_list=op.multi_face_landmarks[0],
                    connections=facemesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                )
                draw.draw_landmarks(
                    image=mesh_window,
                    landmark_list=op.multi_face_landmarks[0],
                    connections=facemesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
                )
                draw.draw_landmarks(
                    image=mesh_window,
                    landmark_list=op.multi_face_landmarks[0],
                    connections=facemesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style()
                )

                #cv2.imshow("Face Mesh", mesh_window)
                #cv2.imshow('MediaPipe Face Mesh', cv2.flip(mesh_window, 1))

                resized_frame1 = cv2.resize(frame, ( int(height/2),width))
                resized_frame2 = cv2.resize(mesh_window, (int(height/2), width))

                frame3 = addFrame(resized_frame1,resized_frame2)

                cv2.imwrite(frame_filename, frame3)

    # Esc to break loop
    if cv2.waitKey(1) == 27:
        video.release()
        cv2.destroyAllWindows()
        break


# Folder containing image frames
input_folder = 'mesh_frames'

# list of image files in the folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg'))]
image_files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # Sort numerically based on frame number

# dimensions of the image
first_image = cv2.imread(os.path.join(input_folder, image_files[0]))
height, width, _ = first_image.shape

# VideoWriter object
output_video_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can try other codecs like 'XVID'
fps = 24  # Set the desired frames per second
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Write each image to the video
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    frame = cv2.imread(image_path)
    out.write(frame)

# Release the VideoWriter
out.release()

# Open the video file for playback
cap = cv2.VideoCapture(output_video_path)

# Play the video
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Break the loop if the video has ended

    #cv2.imshow('Video', frame)
    #if cv2.waitKey(30) & 0xFF == 27:  # Press 'Esc' to exit
        #break

# Release the VideoCapture
cap.release()
cv2.destroyAllWindows()

print(f"Video playback complete.")

